In [1]:
from pathlib import Path
import csv
from itertools import groupby
import h5py
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import random
import matplotlib
from matplotlib import pyplot as plt

# Configuration & Utilities

In [2]:
random.seed(42)
COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]
ANNOTATIONS_PATH = "/media/xtrem/data/experiments/nicolingua-0001-language-id/language-id-annotations/metadata.csv"
FEATURE_DIRS = [
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-z',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-z'
]
# RESULTS_DIR = 'results_005_lang_id_classification'
RESULTS_DIR = 'results_005_lang_id_classification_dropout_0.6'
GPU_ID = 1

In [3]:
annotation_specification = {
    0: {
        'id': 0,
        'label': "maninka",
        'required_tags': set(['ct-speech', 'lng-maninka']),
        'forbidden_tags':  set(['lng-susu', 'lng-pular'])
    },
    1: {
        'id': 1,
        'label': "susu",
        'required_tags': set(['ct-speech', 'lng-susu']),
        'forbidden_tags':  set(['lng-maninka', 'lng-pular'])
    },
    2: {
        'id': 2,
        'label': "pular",
        'required_tags': set(['ct-speech', 'lng-pular']),
        'forbidden_tags':  set(['lng-susu', 'lng-maninka'])
    }
}

In [4]:
bias_category_specification = [
    {
        "category": "utterance",
        "subcategories": [
            {
                "subcategory": "verbal_nod",
                "tags": set(["utt-verbal-nod"])
            },
            {
                "subcategory": "multilingual",
                "tags": set(["utt-multi-lingual", "utt-multi-lingual-named-endity"])
            }
        ]
    },
    {
        "category": "speaker_count",
        "subcategories": [
            {
                "subcategory": "single",
                "tags": set(["spkr-single"])
            },
            {
                "subcategory": "multiple",
                "tags": set(["spkr-mult", "spkr-multi"])
            }
        ]
    },
    {
        "category": "gender",
        "subcategories": [
            {
                "subcategory": "male",
                "tags": set(["spkr-male"])
            },
            {
                "subcategory": "female",
                "tags": set(["spkr-female"])
            },
        ]
    },
    {
        "category": "language",
        "subcategories": [
            {
                "subcategory": "susu",
                "tags": set(["lng-susu"])
            },
            {
                "subcategory": "maninka",
                "tags": set(["lng-maninka"])
            },
            {
                "subcategory": "pular",
                "tags": set(["lng-pular"])
            }
        ]
    },
    {
        "category": "channel",
        "subcategories": [
            {
                "subcategory": "telephone",
                "tags": set(["ct-telephone"])
            },
            {
                "subcategory": "noise",
                "tags": set(["ct-noise"])
            },
            {
                "subcategory": "music",
                "tags": set(["ct-fg-music", "ct-tr-music", "ct-bg-music"])
            }
        ]
    }
]

flat_bias_category_specification = {}

for c in bias_category_specification:
    for sc in c['subcategories']:
        k = f"{c['category']}_{sc['subcategory']}"
        flat_bias_category_specification[k] = sc['tags']

_ = [print(f"{k}: {v}") for k,v in flat_bias_category_specification.items()]

utterance_verbal_nod: {'utt-verbal-nod'}
utterance_multilingual: {'utt-multi-lingual', 'utt-multi-lingual-named-endity'}
speaker_count_single: {'spkr-single'}
speaker_count_multiple: {'spkr-mult', 'spkr-multi'}
gender_male: {'spkr-male'}
gender_female: {'spkr-female'}
language_susu: {'lng-susu'}
language_maninka: {'lng-maninka'}
language_pular: {'lng-pular'}
channel_telephone: {'ct-telephone'}
channel_noise: {'ct-noise'}
channel_music: {'ct-tr-music', 'ct-fg-music', 'ct-bg-music'}


In [5]:
def to_user_friendly_feature_name(fv_name):
    name = fv_name \
        .replace("features-", "") \
        .replace("wav2vec_", "") \
        .replace("average", "avg") \
        .replace("timestep", "T") \
        .replace("c.", "Context") \
        .replace("z.", "Latent")
    return name

# Load annotations

In [6]:
def load_annotations(a_file_path, a_specification):
    with open(ANNOTATIONS_PATH) as f:
        reader = csv.DictReader(f)
        for row in reader:
            tag_set = set([t.strip() for t in row['tags'].split(";")])
            for label in annotation_specification.keys():
                spec = annotation_specification[label]
                if spec['required_tags'].issubset(tag_set):
                    if spec['forbidden_tags'].isdisjoint(tag_set):
                        yield row['file'], label, tag_set
                        break

data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
audio_files, audio_labels, audio_tags = zip(*data)

## Inspect label counts

In [7]:
def inspect_label_counts():
    for label in annotation_specification:
        count = len([l for l in audio_labels if l == label])
        print("{:10} ({}): {}".format(
            annotation_specification[label]['label'],
            label, 
            count
        ))
inspect_label_counts()

maninka    (0): 114
susu       (1): 32
pular      (2): 28


## Balance data

In [8]:
count_per_class = 28
data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
balanced_data = []
for label in annotation_specification:
    balanced_data.extend([d for d in data if d[1] == label][:count_per_class])
audio_files, audio_labels, audio_tags = zip(*balanced_data)

In [9]:
inspect_label_counts()

maninka    (0): 28
susu       (1): 28
pular      (2): 28


## Inspect bias category counts in balanded data

In [10]:
def inspect_bias_category_counts():
    for name, tags in flat_bias_category_specification.items():
        count = len([ts for ts in audio_tags if len(tags.intersection(ts))>0])
        print(name, count, "/", len(audio_tags))
            
inspect_bias_category_counts()

utterance_verbal_nod 48 / 84
utterance_multilingual 21 / 84
speaker_count_single 25 / 84
speaker_count_multiple 58 / 84
gender_male 81 / 84
gender_female 15 / 84
language_susu 28 / 84
language_maninka 28 / 84
language_pular 28 / 84
channel_telephone 27 / 84
channel_noise 21 / 84
channel_music 24 / 84


# Prepare 10 cross validation folds

In [11]:
TRAIN_PERCENT = .6
FOLD_COUNT = 10

n = len(audio_files)
n_train = int(np.ceil(n * .6))
n_test = n - n_train
all_indices = range(n)

cv_folds = {}
train_count_by_index = {i:0 for i in all_indices}
test_count_by_index = {i:0 for i in all_indices}

for fold_index in range(FOLD_COUNT):
    fold_rsampler = np.random.RandomState(seed=fold_index)
    train_index_set = set(fold_rsampler.choice(all_indices, n_train, replace=False))
    test_index_set = set(all_indices).difference(train_index_set)
        
    cv_folds[fold_index] = {
        'train_indices': sorted(list(train_index_set)),
        'test_indices': sorted(list(test_index_set)),
    }


# Load features

In [12]:
def load_features(audio_files, features_input_dir):
    id_list = []
    features_list = []

    for audio_file_name in audio_files:
        feature_file_name = audio_file_name.replace(".wav", ".h5context")
        feature_path = Path(features_input_dir) / feature_file_name
        with h5py.File(feature_path, 'r') as f:
            features_shape = f['info'][1:].astype(int)
            features = np.array(f['features'][:]).reshape(features_shape)
            # features = pool_feature_last_seq(features)
            features_list.append(features)
    return features_list

In [13]:
raw_features = {}
for feature_dir in FEATURE_DIRS:
    feature_name = Path(feature_dir).stem
    raw_features[feature_name] = load_features(audio_files, feature_dir)

## Inspect feature shapes

In [14]:
for feature_name in raw_features.keys():
    print("feature_name: {}. feature shape: {}".format(
        to_user_friendly_feature_name(feature_name),
        raw_features[feature_name][0].shape
    ))

feature_name: c. feature shape: (2998, 512)
feature_name: z. feature shape: (2998, 512)
feature_name: retrained-c. feature shape: (2998, 512)
feature_name: retrained-z. feature shape: (2998, 512)


## Extract feature vectors

In [15]:
def extract_last_timestep_features(raw_features):
    return raw_features[-1, :]

def extract_neuron_average_features(raw_features):
    return np.mean(raw_features, axis=0)

def identity(x):
    return x

feature_extractors = {
    'last_timestep': extract_last_timestep_features,
    'neuron_average': extract_neuron_average_features,
    'raw_features': identity
}

In [16]:
feature_vectors = {}
for feature_name in raw_features.keys():
    for feature_extractor_name in feature_extractors.keys():
        fv_name = f"{feature_name}__{feature_extractor_name}"
        feature_vectors[fv_name] = []
        for f in raw_features[feature_name]:
            feature_vectors[fv_name].append(
                feature_extractors[feature_extractor_name](f)
            )
            
        feature_vectors[fv_name] = np.array(feature_vectors[fv_name])

## Inspect feature vectors

In [17]:
for fvname in feature_vectors.keys():
    print(fvname, feature_vectors[fvname][0].shape)

wav2vec_features-c__last_timestep (512,)
wav2vec_features-c__neuron_average (512,)
wav2vec_features-c__raw_features (2998, 512)
wav2vec_features-z__last_timestep (512,)
wav2vec_features-z__neuron_average (512,)
wav2vec_features-z__raw_features (2998, 512)
retrained-wav2vec_features-c__last_timestep (512,)
retrained-wav2vec_features-c__neuron_average (512,)
retrained-wav2vec_features-c__raw_features (2998, 512)
retrained-wav2vec_features-z__last_timestep (512,)
retrained-wav2vec_features-z__neuron_average (512,)
retrained-wav2vec_features-z__raw_features (2998, 512)


# Classification Models

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_model_summary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [19]:
dropout_p = 0.6
class LangIdCNN_Std1(nn.Module):
    def __init__(self):
        super(LangIdCNN_Std1, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=dropout_p)
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop4 = nn.Dropout(p=dropout_p)
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.lin5 = nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.std(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.std(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.std(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        
        x = self.lin5(v)
        
        return x

class LangIdCNN_Std2(nn.Module):
    def __init__(self):
        super(LangIdCNN_Std2, self).__init__()
        
        self.conv0 = nn.Conv1d(in_channels=512, out_channels=3, kernel_size=1)
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=dropout_p)
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop4 = nn.Dropout(p=dropout_p)
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.lin5 = nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv0(x)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.std(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.std(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.std(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        
        x = self.lin5(v)
        
        return x

class LangIdCNN_Mean1(nn.Module):
    def __init__(self):
        super(LangIdCNN_Mean1, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=dropout_p)
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop4 = nn.Dropout(p=dropout_p)
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.lin5 = nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.mean(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.mean(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.mean(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        
        x = self.lin5(v)
        
        return x
    
    
class LangIdCNN_Mean2(nn.Module):
    def __init__(self):
        super(LangIdCNN_Mean2, self).__init__()
        
        self.conv0 = nn.Conv1d(in_channels=512, out_channels=3, kernel_size=1)
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=dropout_p)
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop4 = nn.Dropout(p=dropout_p)
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.lin5 = nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv0(x)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.mean(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.mean(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.mean(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        
        x = self.lin5(v)
        
        return x

# Train Classification Models

In [20]:
def get_data_for_fold(fold_id, feature_name, batch_size):
    train_indices = cv_folds[fold_id]['train_indices']
    test_indices = cv_folds[fold_id]['test_indices']    

    train_x = np.take(raw_features[feature_name], train_indices, axis=0)
    train_y = np.take(audio_labels, train_indices, axis=0)
    train_tags = np.take(audio_tags, train_indices, axis=0)
    train_bias_category_labels = {}
    for category, tags in flat_bias_category_specification.items():
        train_bias_category_labels[category] = [1 if len(tags.intersection(ts))>0 else 0 for ts in train_tags]

    test_x = np.take(raw_features[feature_name], test_indices, axis=0)
    test_y = np.take(audio_labels, test_indices, axis=0)
    test_tags = np.take(audio_tags, test_indices, axis=0)
    test_bias_category_labels = {}
    for category, tags in flat_bias_category_specification.items():
        test_bias_category_labels[category] = [1 if len(tags.intersection(ts))>0 else 0 for ts in test_tags]
        
    
    return train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels

    
def get_loaders_for_fold(fold_id, feature_name, batch_size):
    
    train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels = \
        get_data_for_fold(fold_id, feature_name, batch_size)
    
    
    
    train_dataset = TensorDataset(
        torch.tensor(train_x), 
        torch.tensor(train_y)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    test_dataset = TensorDataset(
        torch.tensor(test_x), 
        torch.tensor(test_y)
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader, train_bias_category_labels, test_bias_category_labels


def get_predictions_for_logits(logits):
    probs = F.softmax(logits, dim=1)
    return torch.argmax(probs, dim=1)

In [21]:
def train_on_fold(model_class, fold_id, feature_name, batch_size, epochs):
    device = torch.device(f"cuda:{GPU_ID}")
    torch.manual_seed(0)
    results = {}
    
    model = model_class().to(device)

    train_loader, test_loader, train_bias_category_labels, test_bias_category_labels = get_loaders_for_fold(fold_id, feature_name, batch_size)

    print(summary(model_class(), torch.zeros((10, 2998, 512)), show_input=False))

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0
        pred_train_classes = []
        true_train_classes = []

        for batch_idx, (x, y) in enumerate(train_loader):
            x = x.to(device)
            y = y.to(device)
            
            optimizer.zero_grad()
            outputs = model(x)
            pred_train_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )
            true_train_classes.extend(y.cpu().numpy())
            loss = criterion(outputs, y)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_n = len(true_train_classes)
        
        train_loss = train_loss/len(train_loader)
        train_acc = sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes)
        train_acc_by_bais_category = {
            category: sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes, sample_weight=sw)
            for category, sw in train_bias_category_labels.items()
        }
        


        pred_test_classes = []
        true_test_classes = []
        model.eval()
        test_loss = 0
        for batch_idx, (x, y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            
            outputs = model(x)

            pred_test_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )

            true_test_classes.extend(y.cpu().numpy())

            loss = criterion(outputs, y)
            test_loss += loss.item()


        test_n = len(true_test_classes)
        
        test_loss = test_loss / len(test_loader)
        test_acc = sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes)
        test_acc_by_bais_category = {
            category: sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes, sample_weight=sw)
            for category, sw in test_bias_category_labels.items()
        }

        if epoch%10==0:
            print(f"Epoch: {epoch}. Train Loss: {train_loss:0.4}. Test Loss: {test_loss:0.4}. Train Acc: {train_acc:0.4}. Test Acc:{test_acc:0.4}")
            
        results[epoch] = {
            'epoch': epoch,
            'train_loss': train_loss,
            'test_loss': test_loss,
            'train_acc': train_acc,
            'test_acc': test_acc,
            'train_n': train_n,
            'test_n': test_n,
            
        }
        
        for c in train_acc_by_bais_category:
            results[epoch][f"train_acc_{c}"] = train_acc_by_bais_category[c]
            results[epoch][f"train_n_{c}"] = int(np.sum(train_bias_category_labels[c]))
            
        for c in test_acc_by_bais_category:
            results[epoch][f"test_acc_{c}"] = test_acc_by_bais_category[c]
            results[epoch][f"test_n_{c}"] = int(np.sum(test_bias_category_labels[c]))
        
    del model
    return results
    

In [22]:
import csv
from pathlib import Path

def save_results(model_name, all_folds_results):
    for result_entry in all_folds_results:
        feature_name = result_entry['feature_name']
        fold_index = result_entry['fold_index']
        
        Path(RESULTS_DIR).mkdir(exist_ok=True)
        fname = f"{RESULTS_DIR}/{model_name}/{feature_name}_{fold_index}.csv"
        Path(fname).parent.mkdir(parents=True, exist_ok=True)
        with open(fname, 'w') as f:
            fieldnames = sorted(result_entry['epochs'][1].keys())
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            writer.writeheader()
            
            for epoch in sorted(result_entry['epochs'].keys()):

                writer.writerow(result_entry['epochs'][epoch])

In [23]:
model_classes = [
    # LangIdCNN_Std1,
    # LangIdCNN_Mean1,
    # LangIdCNN_Std2,
    LangIdCNN_Mean2
]

for model_class in model_classes:
    all_folds_results = []
    for fold_index in cv_folds:
        
        if fold_index<5:
            continue
            
        for feature_name in raw_features:
            print(f"{model_class.__name__} using {feature_name} on fold#{fold_index}")
            resutls = train_on_fold(model_class, fold_index, feature_name, batch_size=100, epochs=1000)
            all_folds_results.append({
                'fold_index': fold_index,
                'feature_name': feature_name,
                'epochs': resutls
            })
            save_results(model_class.__name__, all_folds_results)

LangIdCNN_Mean2 using wav2vec_features-c on fold#5
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 3, 2998]           1,539           1,539
          Conv1d-2       [10, 1, 2996]              10              10
         Dropout-3       [10, 1, 2996]               0               0
       AvgPool1d-4       [10, 1, 1498]               0               0
          Conv1d-5       [10, 3, 1496]              12              12
         Dropout-6       [10, 3, 1496]               0               0
       AvgPool1d-7        [10, 3, 748]               0               0
          Conv1d-8        [10, 3, 746]              30              30
         Dropout-9        [10, 3, 746]               0               0
      AvgPool1d-10        [10, 3, 373]               0               0
         Conv1d-11        [10, 3, 371]              30              30
        Dropout-12       

Epoch: 820. Train Loss: 0.01442. Test Loss: 5.197. Train Acc: 1.0. Test Acc:0.4545
Epoch: 830. Train Loss: 0.01173. Test Loss: 5.216. Train Acc: 1.0. Test Acc:0.4242
Epoch: 840. Train Loss: 0.01003. Test Loss: 5.227. Train Acc: 1.0. Test Acc:0.4242
Epoch: 850. Train Loss: 0.009134. Test Loss: 5.325. Train Acc: 1.0. Test Acc:0.4545
Epoch: 860. Train Loss: 0.01302. Test Loss: 5.346. Train Acc: 1.0. Test Acc:0.4545
Epoch: 870. Train Loss: 0.008517. Test Loss: 5.358. Train Acc: 1.0. Test Acc:0.4242
Epoch: 880. Train Loss: 0.006365. Test Loss: 5.412. Train Acc: 1.0. Test Acc:0.4545
Epoch: 890. Train Loss: 0.008248. Test Loss: 5.396. Train Acc: 1.0. Test Acc:0.4242
Epoch: 900. Train Loss: 0.0154. Test Loss: 5.478. Train Acc: 1.0. Test Acc:0.4545
Epoch: 910. Train Loss: 0.007994. Test Loss: 5.498. Train Acc: 1.0. Test Acc:0.4545
Epoch: 920. Train Loss: 0.005917. Test Loss: 5.484. Train Acc: 1.0. Test Acc:0.4242
Epoch: 930. Train Loss: 0.004754. Test Loss: 5.546. Train Acc: 1.0. Test Acc:0.424

Epoch: 620. Train Loss: 0.1051. Test Loss: 2.529. Train Acc: 0.9608. Test Acc:0.5455
Epoch: 630. Train Loss: 0.1205. Test Loss: 2.58. Train Acc: 0.9412. Test Acc:0.5455
Epoch: 640. Train Loss: 0.08911. Test Loss: 2.639. Train Acc: 0.9804. Test Acc:0.5455
Epoch: 650. Train Loss: 0.1103. Test Loss: 2.705. Train Acc: 0.9804. Test Acc:0.5455
Epoch: 660. Train Loss: 0.07192. Test Loss: 2.761. Train Acc: 0.9804. Test Acc:0.5152
Epoch: 670. Train Loss: 0.1239. Test Loss: 2.81. Train Acc: 0.9804. Test Acc:0.5152
Epoch: 680. Train Loss: 0.06048. Test Loss: 2.844. Train Acc: 0.9804. Test Acc:0.5455
Epoch: 690. Train Loss: 0.04847. Test Loss: 2.948. Train Acc: 1.0. Test Acc:0.5455
Epoch: 700. Train Loss: 0.09487. Test Loss: 3.017. Train Acc: 0.9608. Test Acc:0.5455
Epoch: 710. Train Loss: 0.04844. Test Loss: 3.053. Train Acc: 1.0. Test Acc:0.5455
Epoch: 720. Train Loss: 0.03891. Test Loss: 3.078. Train Acc: 1.0. Test Acc:0.5455
Epoch: 730. Train Loss: 0.07007. Test Loss: 3.118. Train Acc: 0.9804.

Epoch: 420. Train Loss: 0.1036. Test Loss: 1.113. Train Acc: 1.0. Test Acc:0.5152
Epoch: 430. Train Loss: 0.08874. Test Loss: 1.142. Train Acc: 1.0. Test Acc:0.5152
Epoch: 440. Train Loss: 0.07524. Test Loss: 1.154. Train Acc: 1.0. Test Acc:0.5455
Epoch: 450. Train Loss: 0.08166. Test Loss: 1.22. Train Acc: 1.0. Test Acc:0.5152
Epoch: 460. Train Loss: 0.06711. Test Loss: 1.224. Train Acc: 1.0. Test Acc:0.5152
Epoch: 470. Train Loss: 0.06332. Test Loss: 1.272. Train Acc: 1.0. Test Acc:0.5152
Epoch: 480. Train Loss: 0.05544. Test Loss: 1.295. Train Acc: 1.0. Test Acc:0.5455
Epoch: 490. Train Loss: 0.05848. Test Loss: 1.308. Train Acc: 1.0. Test Acc:0.5455
Epoch: 500. Train Loss: 0.04653. Test Loss: 1.363. Train Acc: 1.0. Test Acc:0.5152
Epoch: 510. Train Loss: 0.03932. Test Loss: 1.415. Train Acc: 1.0. Test Acc:0.5152
Epoch: 520. Train Loss: 0.03707. Test Loss: 1.445. Train Acc: 1.0. Test Acc:0.5152
Epoch: 530. Train Loss: 0.03171. Test Loss: 1.461. Train Acc: 1.0. Test Acc:0.5152
Epoch:

Epoch: 230. Train Loss: 0.5463. Test Loss: 1.329. Train Acc: 0.6863. Test Acc:0.3333
Epoch: 240. Train Loss: 0.5231. Test Loss: 1.357. Train Acc: 0.6863. Test Acc:0.3333
Epoch: 250. Train Loss: 0.5261. Test Loss: 1.395. Train Acc: 0.6667. Test Acc:0.3636
Epoch: 260. Train Loss: 0.5088. Test Loss: 1.431. Train Acc: 0.7059. Test Acc:0.303
Epoch: 270. Train Loss: 0.5097. Test Loss: 1.46. Train Acc: 0.6863. Test Acc:0.303
Epoch: 280. Train Loss: 0.5133. Test Loss: 1.509. Train Acc: 0.6471. Test Acc:0.2727
Epoch: 290. Train Loss: 0.4959. Test Loss: 1.545. Train Acc: 0.7451. Test Acc:0.303
Epoch: 300. Train Loss: 0.487. Test Loss: 1.555. Train Acc: 0.7647. Test Acc:0.2727
Epoch: 310. Train Loss: 0.4902. Test Loss: 1.581. Train Acc: 0.6667. Test Acc:0.303
Epoch: 320. Train Loss: 0.4803. Test Loss: 1.614. Train Acc: 0.7059. Test Acc:0.3939
Epoch: 330. Train Loss: 0.4788. Test Loss: 1.638. Train Acc: 0.7255. Test Acc:0.3939
Epoch: 340. Train Loss: 0.4724. Test Loss: 1.628. Train Acc: 0.7451. Te

...

Epoch: 1000. Train Loss: 0.004528. Test Loss: 2.514. Train Acc: 1.0. Test Acc:0.697
LangIdCNN_Mean2 using wav2vec_features-c on fold#9
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 3, 2998]           1,539           1,539
          Conv1d-2       [10, 1, 2996]              10              10
         Dropout-3       [10, 1, 2996]               0               0
       AvgPool1d-4       [10, 1, 1498]               0               0
          Conv1d-5       [10, 3, 1496]              12              12
         Dropout-6       [10, 3, 1496]               0               0
       AvgPool1d-7        [10, 3, 748]               0               0
          Conv1d-8        [10, 3, 746]              30              30
         Dropout-9        [10, 3, 746]               0               0
      AvgPool1d-10        [10, 3, 373]               0               0
         Con

Epoch: 810. Train Loss: 0.008464. Test Loss: 4.98. Train Acc: 1.0. Test Acc:0.4242
Epoch: 820. Train Loss: 0.009237. Test Loss: 5.067. Train Acc: 1.0. Test Acc:0.4242
Epoch: 830. Train Loss: 0.01131. Test Loss: 4.978. Train Acc: 1.0. Test Acc:0.4242
Epoch: 840. Train Loss: 0.01133. Test Loss: 5.025. Train Acc: 1.0. Test Acc:0.4242
Epoch: 850. Train Loss: 0.0115. Test Loss: 5.138. Train Acc: 1.0. Test Acc:0.4242
Epoch: 860. Train Loss: 0.01137. Test Loss: 5.28. Train Acc: 1.0. Test Acc:0.4242
Epoch: 870. Train Loss: 0.01932. Test Loss: 5.489. Train Acc: 1.0. Test Acc:0.4242
Epoch: 880. Train Loss: 0.009712. Test Loss: 5.548. Train Acc: 1.0. Test Acc:0.3939
Epoch: 890. Train Loss: 0.006971. Test Loss: 5.327. Train Acc: 1.0. Test Acc:0.4242
Epoch: 900. Train Loss: 0.01307. Test Loss: 5.264. Train Acc: 1.0. Test Acc:0.4242
Epoch: 910. Train Loss: 0.02176. Test Loss: 5.337. Train Acc: 1.0. Test Acc:0.4242
Epoch: 920. Train Loss: 0.01169. Test Loss: 5.328. Train Acc: 1.0. Test Acc:0.4242
Epo

Epoch: 610. Train Loss: 0.1713. Test Loss: 1.461. Train Acc: 0.9608. Test Acc:0.6364
Epoch: 620. Train Loss: 0.1281. Test Loss: 1.542. Train Acc: 0.9608. Test Acc:0.6364
Epoch: 630. Train Loss: 0.1336. Test Loss: 1.569. Train Acc: 0.9804. Test Acc:0.6364
Epoch: 640. Train Loss: 0.1239. Test Loss: 1.605. Train Acc: 0.9804. Test Acc:0.6364
Epoch: 650. Train Loss: 0.1334. Test Loss: 1.662. Train Acc: 0.9412. Test Acc:0.6667
Epoch: 660. Train Loss: 0.1141. Test Loss: 1.717. Train Acc: 0.9804. Test Acc:0.6364
Epoch: 670. Train Loss: 0.09388. Test Loss: 1.77. Train Acc: 1.0. Test Acc:0.697
Epoch: 680. Train Loss: 0.1067. Test Loss: 1.881. Train Acc: 0.9804. Test Acc:0.6667
Epoch: 690. Train Loss: 0.07815. Test Loss: 1.868. Train Acc: 1.0. Test Acc:0.6667
Epoch: 700. Train Loss: 0.06509. Test Loss: 1.839. Train Acc: 1.0. Test Acc:0.6667
Epoch: 710. Train Loss: 0.06276. Test Loss: 1.947. Train Acc: 1.0. Test Acc:0.6667
Epoch: 720. Train Loss: 0.05305. Test Loss: 2.032. Train Acc: 1.0. Test Acc

Epoch: 410. Train Loss: 0.05253. Test Loss: 0.5512. Train Acc: 1.0. Test Acc:0.7879
Epoch: 420. Train Loss: 0.04955. Test Loss: 0.5599. Train Acc: 1.0. Test Acc:0.7879
Epoch: 430. Train Loss: 0.04272. Test Loss: 0.5791. Train Acc: 1.0. Test Acc:0.7879
Epoch: 440. Train Loss: 0.03903. Test Loss: 0.5946. Train Acc: 1.0. Test Acc:0.7879
Epoch: 450. Train Loss: 0.03181. Test Loss: 0.6627. Train Acc: 1.0. Test Acc:0.8182
Epoch: 460. Train Loss: 0.02952. Test Loss: 0.6109. Train Acc: 1.0. Test Acc:0.7879
Epoch: 470. Train Loss: 0.03439. Test Loss: 0.5957. Train Acc: 1.0. Test Acc:0.7879
Epoch: 480. Train Loss: 0.0203. Test Loss: 0.6093. Train Acc: 1.0. Test Acc:0.7879
Epoch: 490. Train Loss: 0.01803. Test Loss: 0.6087. Train Acc: 1.0. Test Acc:0.7879
Epoch: 500. Train Loss: 0.02226. Test Loss: 0.6365. Train Acc: 1.0. Test Acc:0.7879
Epoch: 510. Train Loss: 0.02285. Test Loss: 0.6394. Train Acc: 1.0. Test Acc:0.7879
Epoch: 520. Train Loss: 0.01781. Test Loss: 0.6477. Train Acc: 1.0. Test Acc:

Epoch: 210. Train Loss: 0.5998. Test Loss: 0.8601. Train Acc: 0.6863. Test Acc:0.6364
Epoch: 220. Train Loss: 0.5699. Test Loss: 0.8254. Train Acc: 0.6667. Test Acc:0.6364
Epoch: 230. Train Loss: 0.5481. Test Loss: 0.8498. Train Acc: 0.6863. Test Acc:0.6364
Epoch: 240. Train Loss: 0.5265. Test Loss: 0.8199. Train Acc: 0.7843. Test Acc:0.6364
Epoch: 250. Train Loss: 0.5157. Test Loss: 0.8561. Train Acc: 0.7843. Test Acc:0.6061
Epoch: 260. Train Loss: 0.4961. Test Loss: 0.8223. Train Acc: 0.8235. Test Acc:0.6061
Epoch: 270. Train Loss: 0.4702. Test Loss: 0.8742. Train Acc: 0.8824. Test Acc:0.6061
Epoch: 280. Train Loss: 0.448. Test Loss: 0.7906. Train Acc: 0.8824. Test Acc:0.6061
Epoch: 290. Train Loss: 0.4212. Test Loss: 0.8282. Train Acc: 0.8824. Test Acc:0.6061
Epoch: 300. Train Loss: 0.4062. Test Loss: 0.7713. Train Acc: 0.902. Test Acc:0.6667
Epoch: 310. Train Loss: 0.3807. Test Loss: 0.759. Train Acc: 0.902. Test Acc:0.6364
Epoch: 320. Train Loss: 0.3509. Test Loss: 0.7444. Train A

In [24]:
for i in cv_folds:
    if i <5:
        continue
    print(i)



5
6
7
8
9
